<a href="https://colab.research.google.com/github/BorisAnimal/NLP-final-project/blob/lstm/main%20(bert%20-%20colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
tf.version.VERSION

'1.15.2'

In [3]:
! pip install 'kashgari>=1.0.0,<2.0.0'

In [4]:
! wget https://sites.google.com/site/offensevalsharedtask/olid/OLIDv1.0.zip
! mkdir data 
! unzip OLIDv1.0.zip -d data

--2020-05-07 16:27:28--  https://sites.google.com/site/offensevalsharedtask/olid/OLIDv1.0.zip
Resolving sites.google.com (sites.google.com)... 172.217.14.78, 2607:f8b0:4007:803::200e
Connecting to sites.google.com (sites.google.com)|172.217.14.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://sites.google.com/site/offensevalsharedtask/olid/OLIDv1.0.zip?attredirects=0 [following]
--2020-05-07 16:27:28--  https://sites.google.com/site/offensevalsharedtask/olid/OLIDv1.0.zip?attredirects=0
Reusing existing connection to sites.google.com:443.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://ef80a887-a-62cb3a1a-s-sites.googlegroups.com/site/offensevalsharedtask/olid/OLIDv1.0.zip?attachauth=ANoY7cqVNZDfbMJGrHq0bCLZOvzSZjf31YX-bubsxQmWDxurr36KDG2tLkVa44l70_mtDrk5Oz2RPHDztvxeE-WrXIUmGsmlJTa_no8VMZSYHVwxeiNzfOXncvDxOvuH7drfyo_BPnPYwhUdnQW-2dLIMEBSHkiQr6bwna262Kaq7C7MtOJZiSeybJGzZmbzaSgPRDXp1iJCKSbyLTu_kZRrgqzR9IuG

In [5]:
! wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-4_H-512_A-8.zip
! mkdir models
! unzip uncased_L-4_H-512_A-8.zip -d models

--2020-05-07 16:51:32--  https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-4_H-512_A-8.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.5.208, 2607:f8b0:4007:801::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.5.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107814641 (103M) [application/zip]
Saving to: ‘uncased_L-4_H-512_A-8.zip.1’

uncased_L-4_H-512_A 100%[===================>] 102.82M   168MB/s    in 0.6s    

2020-05-07 16:51:33 (168 MB/s) - ‘uncased_L-4_H-512_A-8.zip.1’ saved [107814641/107814641]

mkdir: cannot create directory ‘models’: File exists
Archive:  uncased_L-4_H-512_A-8.zip
replace models/bert_model.ckpt.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [6]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import kashgari
from kashgari.tasks.classification import BiGRU_Model
from kashgari.embeddings import BERTEmbedding

import logging
logging.basicConfig(level='DEBUG')

In [7]:
train = pd.read_csv('data/olid-training-v1.0.tsv', sep='\t', index_col='id')[['tweet', 'subtask_a']]
train.columns = ['text', 'label']
train.head(2)

,text,label
id,,
86426,@USER She should ask a few native Americans wh...,OFF
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF


In [8]:
bert_embed = BERTEmbedding('models',
                           task=kashgari.CLASSIFICATION,
                           sequence_length=128)

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [0]:
tokenizer = bert_embed.tokenizer
x = train.apply(lambda r: tokenizer.tokenize(r['text']), axis=1)
y = train['label']

# train val split
train_x, valid_x, train_y, valid_y = train_test_split(x.values.tolist(),
                                                      y.values.tolist(),
                                                      test_size=0.2, random_state=42)

In [10]:

checkpoint_path = "models/best.pth"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 mode='max', monitor='val_acc',
                                                 save_best_only=True,
                                                 verbose=1)

model = BiGRU_Model(bert_embed)
model.fit(train_x, train_y, valid_x, valid_y, callbacks=[cp_callback])

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 128)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 512), ( 15627264    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 512)     1024        Input-Segment[0][0]              
____________________________________________________________________________________________

### Test data

In [0]:
testX = pd.read_csv("data/testset-levela.tsv", sep='\t', index_col='id')['tweet'].apply(lambda r: tokenizer.tokenize(r)).values
testY = pd.read_csv("data/labels-levela.csv", index_col=0, header=None)[1].values

In [0]:
y_pred = model.predict(testX, batch_size=8)
y_pred = np.array(y_pred)

In [33]:
acc = (sum(y_pred == testY)) / len(y_pred)

print(f"Accuracy: {acc :.4f}")

Accuracy: 0.8070


In [0]:
# type(y_pred), type(testY)
# y_pred.shape, testY.shape

uniq = set(y)
mapping = dict([(k,v) for v,k in enumerate(uniq)])
transform = lambda y: np.array([mapping[i] for i in y])
y_pred = transform(y_pred)
testY = transform(testY)

In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy score: {:.4f}".format(accuracy_score(testY, y_pred)))
print("Precision score: {:.4f}".format(precision_score(testY, y_pred)))
print("Recall score: {:.4f}".format(recall_score(testY, y_pred)))

Accuracy score: 0.8070
Precision score: 0.8547
Recall score: 0.8823
